In [ ]:
"""
Comprehensive Multi-File Strategy Analyzer
"""

from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple, Any
from enum import Enum
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from matplotlib.backends.backend_pdf import PdfPages
from google.colab import drive
import os
import re
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")


# ============================================================
# Configuration and Data Models
# ============================================================


class Config:
    """Configuration singleton for application settings"""

    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance.initialize()
        return cls._instance

    def initialize(self):
        self.TOTAL_KALICUBE_PAGES = 1841
        self.BASE_PATH = "/content/drive/MyDrive/WebKnoGraph"
        self.OUTPUT_PATH = "/content/drive/MyDrive/WebKnoGraph"
        self.DPI = 300
        self.FIGURE_SIZE = (20, 12)
        self.STYLE = "seaborn-v0_8-whitegrid"


class FileType(Enum):
    """Enumeration for file types"""

    AUTOMATIC = "automatic"
    EXPERT = "expert"
    OTHER = "other"


class SubCategory(Enum):
    """Enumeration for sub-categories"""

    BA = "ba"
    REAL_WWW = "real_www"
    UNKNOWN = "unknown"


@dataclass
class FileInfo:
    """Data class for file information"""

    full_path: str
    relative_path: str
    filename: str
    folder: str
    size_kb: float
    modified: str
    type: FileType
    sub_category: SubCategory


# ============================================================
# File Management
# ============================================================


class FileManager:
    """Handles file discovery and loading operations"""

    def __init__(self):
        self.config = Config()
        self.mount_drive()

    def mount_drive(self):
        """Mount Google Drive"""
        print("Mounting Google Drive...")
        drive.mount("/content/drive", force_remount=True)
        print("✓ Google Drive mounted successfully\n")

    def categorize_file(self, file_path: str) -> Tuple[FileType, SubCategory]:
        """Categorize file based on path and name"""
        path_lower = file_path.lower()

        # Determine type
        if "automatic" in path_lower:
            type_category = FileType.AUTOMATIC
        elif "expert" in path_lower:
            type_category = FileType.EXPERT
        else:
            type_category = FileType.OTHER

        # Determine sub-category
        if "ba_" in path_lower or "/ba" in path_lower:
            sub_category = SubCategory.BA
        elif "real_www" in path_lower or "realwww" in path_lower:
            sub_category = SubCategory.REAL_WWW
        else:
            sub_category = SubCategory.UNKNOWN

        return type_category, sub_category

    def find_multi_files(self) -> List[FileInfo]:
        """Find and categorize all multi*.csv files"""
        all_files = []

        print("🔍 Searching for all 'multi*.csv' files in WebKnoGraph...")
        print("=" * 60)

        for root, dirs, files in os.walk(self.config.BASE_PATH):
            dirs[:] = [d for d in dirs if not d.startswith(".") and "Trash" not in d]

            for file in files:
                if file.lower().startswith("multi") and file.lower().endswith(".csv"):
                    full_path = os.path.join(root, file)
                    relative_path = full_path.replace(self.config.BASE_PATH + "/", "")

                    file_size = os.path.getsize(full_path)
                    mod_time = os.path.getmtime(full_path)
                    mod_date = datetime.fromtimestamp(mod_time).strftime(
                        "%Y-%m-%d %H:%M"
                    )

                    type_cat, sub_cat = self.categorize_file(relative_path)

                    file_info = FileInfo(
                        full_path=full_path,
                        relative_path=relative_path,
                        filename=file,
                        folder=os.path.basename(os.path.dirname(full_path)),
                        size_kb=file_size / 1024,
                        modified=mod_date,
                        type=type_cat,
                        sub_category=sub_cat,
                    )
                    all_files.append(file_info)

        self._display_found_files(all_files)
        return all_files

    def _display_found_files(self, all_files: List[FileInfo]):
        """Display found files by category"""
        print(f"✓ Found {len(all_files)} total multi*.csv files\n")
        if not all_files:
            return

        df = pd.DataFrame([vars(f) for f in all_files])
        df["type"] = df["type"].apply(lambda x: x.value)
        df["sub_category"] = df["sub_category"].apply(lambda x: x.value)

        for type_cat, group in df.groupby("type"):
            print(f"\n📁 {type_cat.upper()} FILES ({len(group)} files):")
            print("-" * 50)
            for sub_cat, sub_group in group.groupby("sub_category"):
                if not sub_group.empty:
                    print(f"\n  {sub_cat.upper()} ({len(sub_group)} files):")
                    for _, row in sub_group.iterrows():
                        print(f"    • {row.relative_path} ({row.size_kb:.1f}KB)")
        print("\n")


# ============================================================
# Data Processing & Visualization (Simplified and Unified)
# ============================================================


# NEW: The visualizer now works with a unified DataFrame, simplifying the logic.
class UnifiedVisualizer:
    """Handles all visualization and analysis tasks from a single master DataFrame."""

    def __init__(self):
        self.config = Config()
        self.figures = []
        plt.style.use(self.config.STYLE)

    # NEW: A single function to create all relevant plots.
    def create_all_plots(self, df: pd.DataFrame):
        """Generates all analysis plots from the master DataFrame."""
        if df.empty:
            print("⚠ DataFrame is empty. Skipping plot generation.")
            return

        print("\n📊 GENERATING ALL VISUALIZATION PLOTS")
        print("=" * 60)

        self._create_strategy_performance_plots(df)
        self._create_type_comparison_plots(df)
        self._create_range_analysis_plots(df)

        print(f"✓ Created {len(self.figures)} comprehensive figures")
        for i, fig in enumerate(self.figures, 1):
            print(f"✓ Displaying Figure {i}")
            plt.show()

    def _create_strategy_performance_plots(self, df: pd.DataFrame):
        """Figure 1: Deep dive into strategy performance."""
        fig, axes = plt.subplots(2, 3, figsize=(22, 14))
        fig.suptitle(
            "Strategy Performance Analysis (Aggregated Across All Files)",
            fontsize=20,
            fontweight="bold",
        )

        # Plot 1: Mean Efficiency by Strategy
        sns.barplot(
            data=df,
            x="strategy",
            y="efficiency",
            ax=axes[0, 0],
            palette="viridis",
            ci="sd",
            capsize=0.1,
        )
        axes[0, 0].set_title("Strategy Efficiency (Mean ± Std)")
        axes[0, 0].set_ylabel("Mean Efficiency")
        axes[0, 0].tick_params(axis="x", rotation=45)

        # Plot 2: Mean Delta % by Strategy
        sns.barplot(
            data=df,
            x="strategy",
            y="mean_delta_pct",
            ax=axes[0, 1],
            palette="plasma",
            ci="sd",
            capsize=0.1,
        )
        axes[0, 1].set_title("Strategy Mean Delta % (Mean ± Std)")
        axes[0, 1].set_ylabel("Mean Delta %")
        axes[0, 1].tick_params(axis="x", rotation=45)

        # Plot 3: Page Distribution by Strategy
        page_dist_df = df.groupby("strategy")[
            ["avg_pages_up", "avg_pages_down", "avg_pages_neutral"]
        ].mean()
        page_dist_df.plot(
            kind="bar", stacked=True, color=["green", "red", "gray"], ax=axes[0, 2]
        )
        axes[0, 2].set_title("Average Page Distribution by Strategy")
        axes[0, 2].set_ylabel(
            f"Average Pages (Total = {self.config.TOTAL_KALICUBE_PAGES})"
        )
        axes[0, 2].tick_params(axis="x", rotation=45)

        # Plot 4: Risk Score by Strategy
        sns.boxplot(
            data=df, x="strategy", y="risk_score", ax=axes[1, 0], palette="coolwarm"
        )
        axes[1, 0].set_title("Risk Score Distribution by Strategy")
        axes[1, 0].set_ylabel("Risk Score (Down/Up)")
        axes[1, 0].tick_params(axis="x", rotation=45)

        # Plot 5: Movement Ratio by Strategy
        df["movement_ratio"] = df["avg_pages_up"] / (
            df["avg_pages_up"] + df["avg_pages_down"]
        )
        sns.barplot(
            data=df,
            x="strategy",
            y="movement_ratio",
            ax=axes[1, 1],
            palette="crest",
            ci="sd",
            capsize=0.1,
        )
        axes[1, 1].set_title("Movement Ratio (Up / Total Moved)")
        axes[1, 1].set_ylabel("Positive Movement Ratio")
        axes[1, 1].tick_params(axis="x", rotation=45)

        # Plot 6: Top 10 configurations
        top_10 = df.sort_values("efficiency", ascending=False).head(10)
        top_10["label"] = top_10["strategy"] + " @ " + top_10["range"]
        sns.barplot(
            data=top_10, y="label", x="efficiency", ax=axes[1, 2], palette="hot"
        )
        axes[1, 2].set_title("Top 10 Configurations by Efficiency")
        axes[1, 2].set_xlabel("Efficiency")
        axes[1, 2].set_ylabel("")

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        self.figures.append(fig)

    def _create_type_comparison_plots(self, df: pd.DataFrame):
        """Figure 2: Compare Automatic vs. Expert performance, grouped by range."""
        fig, axes = plt.subplots(2, 2, figsize=(20, 12))
        fig.suptitle(
            "Automatic vs. Expert Comparison (Grouped by Connection Range)",
            fontsize=20,
            fontweight="bold",
        )

        # Plot 1: Efficiency grouped by range and type
        sns.boxplot(data=df, x="range", y="efficiency", hue="type", ax=axes[0, 0])
        axes[0, 0].set_title("Efficiency Distribution by Range and Type")

        # Plot 2: Mean Delta % grouped by range and type
        sns.barplot(
            data=df,
            x="range",
            y="mean_delta_pct",
            hue="type",
            ax=axes[0, 1],
            ci="sd",
            capsize=0.1,
        )
        axes[0, 1].set_title("Mean Delta % by Range and Type (Mean ± Std)")

        # Plot 3: Risk Score grouped by range and type
        sns.violinplot(
            data=df, x="range", y="risk_score", hue="type", ax=axes[1, 0], split=True
        )
        axes[1, 0].set_title("Risk Score Distribution by Range and Type")

        # Plot 4: Performance Space
        sns.scatterplot(
            data=df,
            x="efficiency",
            y="mean_delta_pct",
            hue="type",
            style="sub_category",
            s=100,
            ax=axes[1, 1],
        )
        axes[1, 1].set_title("Performance Space: Efficiency vs. Mean Delta")
        axes[1, 1].grid(True)

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        self.figures.append(fig)

    def _create_range_analysis_plots(self, df: pd.DataFrame):
        """Figure 3: Analyze performance across different connection ranges."""
        fig, axes = plt.subplots(2, 2, figsize=(20, 12))
        fig.suptitle("Connection Range Analysis", fontsize=20, fontweight="bold")

        # Plot 1: Range Performance (Mean Efficiency)
        sns.pointplot(
            data=df,
            x="range",
            y="efficiency",
            ax=axes[0, 0],
            capsize=0.1,
            join=False,
            ci="sd",
        )
        axes[0, 0].set_title("Mean Efficiency by Connection Range (Mean ± Std)")
        axes[0, 0].set_ylabel("Mean Efficiency")

        # Plot 2: Configuration Distribution
        range_counts = df["range"].value_counts()
        axes[0, 1].pie(
            range_counts, labels=range_counts.index, autopct="%1.1f%%", startangle=90
        )
        axes[0, 1].set_title("Configuration Distribution by Range")

        # Plot 3: Strategy Heatmap across Ranges
        heatmap_data = df.pivot_table(
            index="strategy", columns="range", values="efficiency", aggfunc="mean"
        )
        sns.heatmap(heatmap_data, ax=axes[1, 0], annot=True, fmt=".4f", cmap="magma")
        axes[1, 0].set_title("Mean Efficiency Heatmap: Strategy vs. Range")

        # Plot 4: Best strategy per range
        best_per_range = df.loc[df.groupby("range")["efficiency"].idxmax()]
        sns.barplot(
            data=best_per_range,
            x="range",
            y="efficiency",
            hue="strategy",
            ax=axes[1, 1],
            dodge=False,
        )
        axes[1, 1].set_title("Best Performing Strategy per Range")
        axes[1, 1].legend(title="Strategy", bbox_to_anchor=(1.05, 1), loc="upper left")

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        self.figures.append(fig)

    def save_figures(self, output_path: str = None):
        """Save all generated figures."""
        if not output_path:
            output_path = self.config.OUTPUT_PATH

        plot_names = ["strategy_performance", "type_comparison", "range_analysis"]

        for fig, name in zip(self.figures, plot_names):
            plot_path = f"{output_path}/comprehensive_{name}.png"
            fig.savefig(plot_path, dpi=self.config.DPI, bbox_inches="tight")
            print(
                f"✓ Saved: {name} -> {plot_path.replace('/content/drive/MyDrive/', '')}"
            )

        pdf_path = f"{output_path}/comprehensive_analysis_all_plots.pdf"
        with PdfPages(pdf_path) as pdf:
            for fig in self.figures:
                pdf.savefig(fig, bbox_inches="tight")
        print(
            f"✓ All plots saved as PDF: {pdf_path.replace('/content/drive/MyDrive/', '')}"
        )


# ============================================================
# Main Orchestrator
# ============================================================


class StrategyAnalyzerApp:
    """Main application orchestrator"""

    def __init__(self):
        self.config = Config()
        self.file_manager = FileManager()
        self.visualizer = (
            UnifiedVisualizer()
        )  # CHANGED: Using the new unified visualizer

    def run(self):
        """Run the complete analysis"""
        print("🚀 Starting Comprehensive Analysis...")

        all_files = self.file_manager.find_multi_files()

        if not all_files:
            print("\n❌ No multi*.csv files found")
            return None

        # NEW: Load and combine all data into a single DataFrame first.
        master_df = self._load_and_combine_data(all_files)

        if master_df.empty:
            print("\n❌ No data could be loaded from the files.")
            return None

        # NEW: Generate plots from the unified DataFrame.
        self.visualizer.create_all_plots(master_df)

        # Optional: Save plots and results
        if self._ask_to_save("plots"):
            self.visualizer.save_figures()

        if self._ask_to_save("results"):
            csv_path = os.path.join(
                self.config.OUTPUT_PATH, "master_analysis_results.csv"
            )
            master_df.to_csv(csv_path, index=False)
            print(
                f"✓ Master results DataFrame saved to: {csv_path.replace('/content/drive/MyDrive/', '')}"
            )

        return master_df

    # NEW: This method unifies all data sources.
    def _load_and_combine_data(self, all_files: List[FileInfo]) -> pd.DataFrame:
        """Loads all CSVs and combines them into a single master DataFrame."""
        print("\n🔄 Loading and combining all result files...")
        all_dfs = []
        for file_info in all_files:
            try:
                df = pd.read_csv(file_info.full_path)
                df["type"] = file_info.type.value
                df["sub_category"] = file_info.sub_category.value
                df["source_file"] = file_info.filename
                all_dfs.append(df)
            except Exception as e:
                print(f"  ⚠ Could not load or process {file_info.filename}: {e}")

        if not all_dfs:
            return pd.DataFrame()

        master_df = pd.concat(all_dfs, ignore_index=True)

        # Calculate derived metrics
        if (
            "avg_pages_up" in master_df.columns
            and "avg_pages_down" in master_df.columns
        ):
            master_df["avg_pages_neutral"] = self.config.TOTAL_KALICUBE_PAGES - (
                master_df["avg_pages_up"] + master_df["avg_pages_down"]
            )
            master_df["risk_score"] = master_df["avg_pages_down"] / (
                master_df["avg_pages_up"] + 0.001
            )

        print(
            f"✓ Combined {len(all_dfs)} files into a single dataset with {len(master_df)} rows."
        )
        return master_df

    def _ask_to_save(self, item: str) -> bool:
        """Generic function to ask user for saving."""
        response = input(f"\n💾 Save all {item} to Drive? (y/n): ").strip().lower()
        return response == "y"


# ============================================================
# Entry Point
# ============================================================


def main():
    """Main entry point"""
    app = StrategyAnalyzerApp()
    results_df = app.run()

    if results_df is not None and not results_df.empty:
        print("\n" + "=" * 60)
        print("✅ ANALYSIS COMPLETE")
        print("=" * 60)
        print("\nKey findings can be observed in the plots above.")
        print("The final DataFrame is returned by this function.")

    return results_df


if __name__ == "__main__":
    master_results = main()